# OUMUAMUA_project

## Proyecto final Bootcamp DataMad0119

### Realizado por: Laura Repullo


En octubre de 2017, un telescopio de la Universidad de Hawai descubrió un misterioso objeto con forma de cigarrillo dando vueltas por el sistema solar; lo bautizaron con el nombre de Oumuamua ("mensajero que llega dese un pasado lejano").

Desde su descubrimiento, los científicos no han podido ponerse de acuerdo para explicar sus características inusuales y su origen preciso. Primero, algunos investigadores dijeron que era un cometa y luego un asteroide, antes de determinar que es el primero de su tipo: una nueva clase de “objetos interestelares”.

En octubre de 2018 Abraham Loeb, profesor y director de astronomía de la Universidad de Harvard, y Shmuel Bialy, alumno del postdoctorado, publicaron un artículo en la revista científica Astrophysical Journal Letters sobre Oumuamua.

“Oumuamua podría ser una sonda totalmente operativa enviada a las inmediaciones de la Tierra por una civilización extraterrestre”, escribieron los investigadores en el trabajo que fue presentado a la revista Astrophysical Journal Letters.

Era la primera vez que la comunidad cientifica hablaba punlicamente de la posibilidad de que pudiera existir vida extraterrestre.

Este proyecto trata de analizar las características de los avistamientos de OVNIs recogidos durante el último siglo en un dataset por la NUFORC (National UFO Reporting Center):

 - Origen de datos:https://www.kaggle.com/NUFORC/ufo-sightings#complete.csv

El dataset tiene las siguientes columnas: 

- datetime: indica el dia y hora del avistamiento.
- city: ciudad desde donde se vio el OVNI
- state: estado desde donde se vio el OVNI
- country: país
- shape: forma del OVNI 
- duration (seconds): duración (segundos)
- duration (hours/min): duración (horas/minutos)
- comments: descrición del avistamiento
- date posted: dia publicado
- latitude: coordinada de latitud del avistamiento
- longitude: coordinada de longitus del avistamiento

Comenzamos el análisis:

## 1. Limpieza de los datos:

Importamos las librerias que vamos a necesitar, tanto para la limpieza como para el aánlisis y tratamiento de los datos:

In [1]:
import pandas as pd
import time
from datetime import date
from datetime import time
import matplotlib.pyplot as plt
import re

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import stem
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
ps = PorterStemmer()

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ironhack")
from geopy.util import get_version
get_version()

[nltk_data] Downloading package stopwords to /home/laura/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/laura/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/laura/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Importamos el dataset "UFO" en formato .csv:

In [ ]:
data = pd.read_csv('./UFO.csv')

Observamos el numero de filas y columnas del dataset:

In [2]:
data.shape

(88875, 14)

In [3]:
data.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')

In [4]:
data.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.9411111,NaN,NaN,NaN
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,NaN,NaN,NaN
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,NaN,NaN,NaN
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.6458333,NaN,NaN,NaN
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.8036111,NaN,NaN,NaN


Analizamos los tipos de datos de las columnas; salvo las 2 ultimas, todos son tipo objeto, y debemos modificar varios:
- 'datetime': lo pasaremos a tipo datetime
- 'duracion (seconds)', 'latitude' y 'longitude', deben ser valores numéricos, para que al trataros los programas los traten correctamente.
 
Tambien unificaremos las mayusculas y minusculas de las columnas 'city', 'country' y 'state' para homogeneizar las columnas.

In [5]:
data.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude                object
Unnamed: 11              object
Unnamed: 12             float64
Unnamed: 13             float64
dtype: object

In [2]:
data['datetime'] = pd.to_datetime(data['datetime'], errors='coerce') #datetime to datetime
data['duration (seconds)'] = pd.to_numeric(data['duration (seconds)'], errors='coerce') #duration (seconds) to float
data['latitude'] = pd.to_numeric(data['latitude'], errors='coerce') #latitud to float
data['longitude'] = pd.to_numeric(data['longitude'], errors='coerce') #longitude to float
data.dtypes
data['city'] = data['city'].str.title()
data['state'] = data['state'].str.upper()
data['country'] = data['country'].str.upper()

Nos fijamos en la columna 'datetime':
 - los datos que tratamos estan comprendidos entre los años 1906 y 2014
 - dividimos la columna entre las columnas 'year', 'date' y 'time', para que más adelante nos permita un mejor análisis:

In [3]:
print(data['datetime'].min())
print(data['datetime'].max())

1906-11-11 00:00:00
2014-05-08 18:45:00


In [4]:
#split in date and time:
data['date'] = [d.date() for d in data['datetime']]
data['year'] = data.datetime.dt.year

data['time'] = data.datetime.dt.time

Comprobamos que columnas tienen valores nulos:
- 'Unnamed: 11 '(88197), 'Unnamed: 12' (88836), 'Unnamed: 13' (88873): casi todas sus filas son valores nulos, asi que las eliminamos.
- es necesario que tambien 'latitud' o 'longitud' tengas valores válidos; eliminamos las filas cuyos valores son nulos.
- tenemos que quitar las latitudes fuera del rango [-90, 90] y las longitudes que esten fuera de [-180, 180].

In [15]:
data1.latitude.max(), data1.latitude.min(), data1.longitude.max(), data1.longitude.min()

(72.7, -82.862752, 178.4419, -176.65805559999998)

In [10]:
data.isnull().sum()

datetime                 1262
city                      196
state                    7396
country                 12401
shape                    3358
duration (seconds)        403
duration (hours/min)     3090
comments                  154
date posted                 3
latitude                  679
longitude                  39
Unnamed: 11             88197
Unnamed: 12             88836
Unnamed: 13             88873
date                     1262
year                     1262
time                     1262
dtype: int64

In [6]:
# eliminamos las columnas que tienen casi todos los valores, nulos:
data1 = data.drop(['Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13','duration (hours/min)'], axis=1).dropna(subset=['latitude', 'longitude'], inplace=False)
#tambien las filas cuyos valores de 'latitud' o 'longitud' son nulos:
data1 = data1[data1['latitude'] != 0.000000]
#tenemos que quitar las latitudes fuera del rango [-90, 90] y las longitudes que esten en [-180, 180]:
data1 = data1[(data1['latitude'].between(-90,90)) & (data1['longitude'].between(-180,180))]

In [7]:
data1.head(143)

,datetime,city,state,country,shape,duration (seconds),comments,date posted,latitude,longitude,date,year,time
0,1949-10-10 20:30:00,San Marcos,TX,US,cylinder,2700.0,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,1949-10-10,1949.0,20:30:00
1,1949-10-10 21:00:00,Lackland Afb,TX,NaN,light,7200.0,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,1949-10-10,1949.0,21:00:00
2,1955-10-10 17:00:00,Chester (Uk/England),NaN,GB,circle,20.0,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,1955-10-10,1955.0,17:00:00
3,1956-10-10 21:00:00,Edna,TX,US,circle,20.0,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,1956-10-10,1956.0,21:00:00
4,1960-10-10 20:00:00,Kaneohe,HI,US,light,900.0,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,1960-10-10,1960.0,20:00:00
5,1961-10-10 19:00:00,Bristol,TN,US,sphere,300.0,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889,1961-10-10,1961.0,19:00:00
6,1965-10-10 21:00:00,Penarth (Uk/Wales),NaN,GB,circle,180.0,penarth uk circle 3mins stayed 30ft above m...,2/14/2006,51.434722,-3.180000,1965-10-10,1965.0,21:00:00
7,1965-10-10 23:45:00,Norwalk,CT,US,disk,1200.0,A bright orange color changing to reddish colo...,10/2/1999,41.117500,-73.408333,1965-10-10,1965.0,23:45:00
8,1966-10-10 20:00:00,Pell City,AL,US,disk,180.0,Strobe Lighted disk shape object observed clos...,3/19/2009,33.586111,-86.286111,1966-10-10,1966.0,20:00:00
9,1966-10-10 21:00:00,Live Oak,FL,US,disk,120.0,Saucer zaps energy from powerline as my pregna...,5/11/2005,30.294722,-82.984167,1966-10-10,1966.0,21:00:00


Estudiamos los valores de 'country': sustituimos valores que están mal registrados, por el codigo de 2 letras según la ISO 3166-1.

No todos los valores son conocidos; asi que completaremos este campo, haciendo request a una API a través de geopy.geocoders, con los valores de latitud y longitud de las columnas correspondientes.

Antes de eso, para tenerlo localizados los valores que debemos buscar, sustitimos los valores nulos por 'other'.

In [14]:
#En la columna Country hay muchos valores erroneos: en un primer momento vamos a sustituir algunos valores:
data1['country'] = data1['country'].replace(['WI', 'FL', 'HI', 'IA', 'KS', 'UT', 'MI', 'WV', 'WY','VT', 'ND', 'NJ','NM', 'NV', 'NY', 'OR', 'TX'], 'US')
data1['country'] = data1['country'].replace(['AB', 'BC', 'MB', 'PQ', 'QC', ' ??) (CANADA)'], 'CA')
data1['country'] = data1['country'].replace(['DC'], 'DK')
data1['country'] = data1['country'].replace([' (CATALUNYA) (SPAIN)'], 'SP')
data1['country'] = data1['country'].replace([' (FINLAND)'], 'FI')
data1['country'] = data1['country'].replace(['N (SWEDEN)',  '(SWEDEN)', 'SAND (SWEDEN)',' (SWEDEN)'], 'SE')
data1['country'] = data1['country'].replace(['Y (NORWAY)',  'NDELAG (NORWAY)'], 'NO')
data1['country'] = data1['country'].replace([' POD RALSKEM (CZECH REPUBLIC)'], 'CZ')
data1['country'] = data1['country'].replace(['A (ARGENTINA)'], 'AR')
data1['country'] = data1['country'].replace(['RITO SANTO STATE) (BRAZIL)', ' CERRO PRIETO.&#44 B. C.'], 'BR')

data1.country.fillna('other',inplace=True)

#data1[data.country.isnull()]
data1.country.unique()

array(['US', 'other', 'GB', 'CA', 'AU', 'DE'], dtype=object)

In [16]:
data1.country.value_counts()

US       70105
other    10554
CA        3243
GB        2038
AU         578
DE         111
Name: country, dtype: int64

## 2. ENRIQUECIMIENTO DE LA BASE DE DATOS:

Como hemos comentado, completaremos la columna 'country' haciendo peticiones a través de geopy.geocoders: 

In [17]:
# Para los datos en los que 'country' es un valor nulo, lo vamos a sustituir por el valor que nos devuelve geolocator al hacer la consulta con los datos de latitud y longitud:

def adress_state (lat, lon):
    coord = str(lat) + ' , '+ str(lon)
    location = geolocator.reverse(coord)
    res = location.raw['address']['state']
    return res
        
def adress_country(lat, lon, country):
    if country != 'other':
        return country
    else: 
        try: 
            coord = str(lat) + ' , '+ str(lon)
            location = geolocator.reverse(coord)
            res2 = location.raw['address']['country']
            return res2
        except: 
            return 'timeout'

#PARA PROBAR LA FUNCION:
print(adress_country(29.883056, -97.941111, 'other'))
      

#DESBLOQUEAR AL FINAL!!!data1['country'] = data1.apply(lambda x: adress_country(x['latitude'], x['longitude'], x['country']), axis = 1)
   # else:
        #i

USA


Guardamos el resultado obtenido en un .csv, al que llamaremos "UFO_modify1.csv".

In [9]:
#data1.to_csv("UFO_modify1.csv")

Seguimos en oumuamua_2.